In [1]:
import sys
from sklearn import svm
import adversaries as ad
from data_reader import input, output, operations
import learners as learner
from metrics import EvasionMetrics
from copy import deepcopy
from data_reader.dataset import EmailDataset

# Library Demo

Here we demonstrate framework testing by simulating an SVM classifier and its  
resulting classification accuracy after an attack from the simple_optimize adversary.

## Initial Learner Training
- choose an initial learner model,
- load the instances for training,
- and wrap the two with the Classifier wrapper. 


In [2]:
from learners import SimpleLearner
learning_model = svm.SVC(probability=True, kernel='linear')
dataset = EmailDataset(path='./data_reader/data/test/100_instance_debug.csv', raw=False)
trainset, testset = dataset.split({'train': 60, 'test': 40})

clf = SimpleLearner(learning_model, trainset)

# instances = input.load_instances('./data_reader/data/test/100_instance_debug')

# clf = Classifier(learning_model, instances)


<class 'scipy.sparse.csr.csr_matrix'>
<class 'numpy.ndarray'>


In [3]:
X, y = trainset.numpy()

In [4]:
y

array([-1.,  1., -1.,  1.,  1.,  1.,  1., -1., -1.,  1., -1.,  1., -1.,
       -1., -1.,  1., -1.,  1.,  1., -1., -1., -1.,  1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,  1.,
        1., -1., -1., -1.,  1., -1.,  1., -1., -1.,  1., -1., -1., -1.,
       -1., -1.,  1., -1., -1.,  1.,  1., -1.])

In [5]:
clf.train() # train the initial learner

In [6]:
len(trainset)

60

## Adversary Attack 
Now we choose an adversary attack strategy to transform the 
original instances, here we choose simple_optimize.

In [7]:
# initialize attacker from the ad package
adv = ad.SimpleOptimize(lambda_val=-100, max_change = 65, learner = clf)
# perform the attack on the data to transform instances
instances_post_attack = adv.attack(trainset)

NotImplementedError: 

### Graph results of pre attack vs. post attack learner 

In [ ]:
# obtain metrics of classification performance both pre- and post- attack
metrics = EvasionMetrics(clf, instances, instances_post_attack)
preAccuracy = metrics.getAccuracy(True)
postAccuracy = metrics.getAccuracy(False)
print ("Pre-attack Accuracy (naive classifier): " + str(preAccuracy))
print ("Post-attack Accuracy: (naive classifier): " + str(postAccuracy))

# plot ROC curve of naive classifier performance after attack
metrics.plotROC(False)

## Learner Response
Here we wrap a defense strategy for the learner to exercise
during the next round of training to become more robust. 

In [ ]:
instances2 = input.load_instances('./data_reader/data/test/100_instance_debug') # load instances
learning_model2 = svm.SVC(probability=True, kernel='linear')
clf2 = Classifier(learning_model2, instances2, ["retraining", adv.clone()]) # could also just pass blank adv instance
clf2.train() # train again with defense strategy 

## Adversary Attack

In [ ]:
# Now we launch the attack on the robust classifier 
adv = ad.SimpleOptimize(lambda_val=-100, max_change = 65, learner = clf2)
instances_post_attack2 = adv.attack(instances2)


### Display metrics after attack on the robust classifier
Here we see the improvements made in accuracy as a result 
of the chosen defense strategy.  Clearly, our learner has 
become more robust to adversary attacks.

In [ ]:
# obtain metrics of classification performance both pre- and post- attack
metrics2 = EvasionMetrics(clf2, instances, instances_post_attack2)
preAccuracy2 = metrics2.getAccuracy(True)
postAccuracy2 = metrics2.getAccuracy(False)
print ("Pre-attack Accuracy (robust classifier): " + str(preAccuracy2))
print ("Post-attack Accuracy: (robust classifier): " + str(postAccuracy2))

metrics2.plotROC(True)